In [3]:
from datetime import datetime, timedelta, timezone
import os
import time
from random import choice
import regex as re
import json
from urllib.parse import parse_qs
###
# Third party imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
import hashlib
from pivottablejs import pivot_ui

In [2]:
# !pip install pandas

In [4]:
def get_random_user_agent() -> str:
    """
    This function returns a user agent signature for a set of predefined user agents.
    The returned user agent is used while scraping which is passed as a header.
    The purpose of using random user agents is to mimic a browser to make sure that
    FixedFloat doesn't identify our scraping script at regular intervals as a bot.
    """

    user_agents = [
        "Mozilla/5.0 (iPad; CPU OS 13_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.77 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (Linux; Android 8.0.0; SM-G955U Build/R16NW) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1",
    ]
    rand_choice = choice([i for i in range(0, len(user_agents))])
    user_agent = user_agents[rand_choice]
    return user_agent

In [5]:
arcl_url = "https://arcl.org/Pages/UI/"
team_stats_url = "TeamStats.aspx?team_id=5666&league_id=4&season_id=56"

user_agent = (
    get_random_user_agent()
)  # The purpose of using random user agents is to mimic a browser to make sure that FF doesn't identify our scraping at regular intervals as a bot
arcl_session = requests.Session()
req = arcl_session.get(
    arcl_url + team_stats_url, headers={"User-Agent": user_agent}, timeout=3
)
soup = BeautifulSoup(req.text, "html.parser")


In [16]:
season = soup.find_all("p", attrs={"class": "centertitle"})[0].text

'Spring 2022 - Div B'

In [6]:
df_team_players_batting = pd.read_html(str(soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView2"})[0]))[0]
df_team_players_batting

,Player,Player_Id,Team,Innings,Runs,Balls,Fours,Sixs,Strike Rate
0,Srinivas Potluri,57069,Master Minds,8,128,86,10,0,148.84
1,Sampath Bodduluri,62464,Master Minds,9,123,124,2,3,99.19
2,Gudapati Avinash,58545,Master Minds,4,122,86,4,0,141.86
3,Sumanth Devi reddy,46547,Master Minds,4,81,95,2,3,85.26
4,Abhi Vyas,62636,Master Minds,10,80,91,6,1,87.91
5,Harsha Gorla,62634,Master Minds,8,64,80,1,2,80.00
6,Sudheer Palempati,61386,Master Minds,7,44,55,2,0,80.00
7,Shankar VK,53152,Master Minds,4,29,22,2,1,131.82
8,Aravind Palempati,62635,Master Minds,3,18,22,1,0,81.82
9,Sankar Gudapati,63327,Master Minds,5,17,32,0,0,53.13


In [337]:
df_team_players_bowling = pd.read_html(str(soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView3"})[0]))[0]
df_team_players_bowling

,Player,Player_Id,Team,Innings,Overs,Maidens,Runs,Wickets,Average,Eco Rate
0,Harsha Gorla,62634,Master Minds,8,21.0,2,105,11,9,5.00
1,Srinivas Potluri,57069,Master Minds,7,24.0,1,102,9,11,4.25
2,Gudapati Avinash,58545,Master Minds,3,10.0,1,39,8,4,3.90
3,Aravind Palempati,62635,Master Minds,4,11.0,0,54,5,10,4.91
4,Shankar VK,53152,Master Minds,4,13.3,2,32,5,6,2.41
5,Vamsi Chanumolu,61383,Master Minds,6,19.6,0,90,4,22,4.59
6,Sudheer Palempati,61386,Master Minds,5,12.3,0,67,3,22,5.45
7,Sankar Gudapati,63327,Master Minds,3,5.0,0,21,2,10,4.20
8,Abhi Vyas,62636,Master Minds,5,10.5,0,56,1,56,5.33
9,Girish Prabhu,62638,Master Minds,1,4.0,0,27,1,27,6.75


In [1]:
get_matchscorecard = soup.find_all("table", attrs={"id": "ctl00_ContentPlaceHolder1_GridView1"})
all_matches_schedule = pd.read_html(str(get_matchscorecard))[0][['Team','Opposition','Match Date','Ground']]
get_matchscorecard = get_matchscorecard[0].find_all("a", attrs={"href": re.compile('.*Match*')})
get_matchscorecard = [y.get("href") for y in get_matchscorecard]

all_matches_schedule['matchscorecard_link'] = get_matchscorecard
all_matches_schedule = all_matches_schedule.rename(columns = {'Match Date' : 'Match_Date'})
all_matches_schedule['Match_Day'] = all_matches_schedule['Match_Date'].apply(lambda x : x.split()[0])
all_matches_schedule['Match_Date'] = all_matches_schedule['Match_Date'].apply(lambda x : x.split()[1])
all_matches_schedule['Match_Date'] = pd.to_datetime(all_matches_schedule['Match_Date'], format='%m/%d/%Y', errors='ignore').dt.strftime('%Y-%m-%d')
# d = all_matches_schedule['Match_Date']
all_matches_schedule['Ground'] = all_matches_schedule['Ground'].apply(lambda x : x.split(' (')[0])
all_matches_schedule

NameError: name 'soup' is not defined

In [339]:
all_matches_schedule.dtypes

Team                   object
Opposition             object
Match_Date             object
Ground                 object
matchscorecard_link    object
Match_Day              object
dtype: object

In [340]:
all_matches_schedule_df_json = eval(all_matches_schedule.to_json(orient='records'))
all_matches_schedule_df_json[0:3]

[{'Team': 'Master Minds',
  'Opposition': 'Dabangg Regiment',
  'Match_Date': '2022-03-26',
  'Ground': 'GrassLawn Park-Softball Field 1',
  'matchscorecard_link': 'MatchScorecard.aspx?match_id=19237&league_id=4&season_id=56',
  'Match_Day': 'Saturday'},
 {'Team': 'Master Minds',
  'Opposition': 'BATMEN RISES',
  'Match_Date': '2022-04-16',
  'Ground': 'GrassLawn Park-Softball Field 1',
  'matchscorecard_link': 'MatchScorecard.aspx?match_id=19315&league_id=4&season_id=56',
  'Match_Day': 'Saturday'},
 {'Team': 'Master Minds',
  'Opposition': 'BATMEN',
  'Match_Date': '2022-05-01',
  'Ground': 'North SeaTac Park',
  'matchscorecard_link': 'MatchScorecard.aspx?match_id=19410&league_id=4&season_id=56',
  'Match_Day': 'Sunday'}]

In [341]:
df_player_batting_scores = pd.DataFrame()
df_player_bowling_scores = pd.DataFrame()

for match_payload in all_matches_schedule_df_json:
#     # print(arcl_url + matchresult_url)
    matchresult_url = match_payload['matchscorecard_link']
    match_date = match_payload['Match_Date']
    opposition = match_payload['Opposition']
    ground = match_payload['Ground']
    matchresult_html = pd.read_html(arcl_url + matchresult_url)
    match_data_json = json.dumps(parse_qs(matchresult_url.split('?')[1]))
    match_id = json.loads(match_data_json)['match_id'][0]
    league_id = json.loads(match_data_json)['league_id'][0]
    season_id = json.loads(match_data_json)['season_id'][0]
    
    for each_batting_scores_table_id in [1,3]:
        try:
            tmp_df_player_batting_scores_per_match = matchresult_html[each_batting_scores_table_id][0:-3]
            tmp_df_player_batting_scores_per_match['match_id'] = match_id
            tmp_df_player_batting_scores_per_match['league_id'] = league_id
            tmp_df_player_batting_scores_per_match['season_id'] = season_id
            tmp_df_player_batting_scores_per_match['match_date'] = match_date
            tmp_df_player_batting_scores_per_match['opposition'] = opposition
            tmp_df_player_batting_scores_per_match['ground'] = ground

            df_player_batting_scores = pd.concat([df_player_batting_scores,tmp_df_player_batting_scores_per_match])
        except Exception as e:
            print(e)
            
    for each_bowling_scores_table_id in [2,4]:
        try:
            tmp_df_player_bowling_scores_per_match = matchresult_html[each_bowling_scores_table_id][0:-3]
            tmp_df_player_bowling_scores_per_match['match_id'] = match_id
            tmp_df_player_bowling_scores_per_match['league_id'] = league_id
            tmp_df_player_bowling_scores_per_match['season_id'] = season_id
            tmp_df_player_bowling_scores_per_match['match_date'] = match_date
            tmp_df_player_bowling_scores_per_match['opposition'] = opposition
            tmp_df_player_bowling_scores_per_match['ground'] = ground

            df_player_bowling_scores = pd.concat([df_player_bowling_scores,tmp_df_player_bowling_scores_per_match])
        except Exception as e:
            print(e)
        # time.sleep(1)

/var/folders/90/v8hnl2t9621ck6lryqcpq4gc0000gn/T/ipykernel_83387/1365777697.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df_player_batting_scores_per_match['match_id'] = match_id
/var/folders/90/v8hnl2t9621ck6lryqcpq4gc0000gn/T/ipykernel_83387/1365777697.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df_player_batting_scores_per_match['league_id'] = league_id
/var/folders/90/v8hnl2t9621ck6lryqcpq4gc0000gn/T/ipykernel_83387/1365777697.py:21: SettingWithCopyWarning: 
A value is trying to

list index out of range
list index out of range
list index out of range
list index out of range


In [342]:
df_player_batting_scores.head()

,Batsman,How_out,Fielder,Bowler,Sixs,Fours,Runs,Balls,match_id,league_id,season_id,match_date,opposition,ground
0,Kanna Thirunarayanan,bowled,NaN,Harsha Gorla,NaN,NaN,1,5.0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
1,Rajat Mahajan,not out,NaN,NaN,NaN,1.0,38,38.0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
2,Priyank Mundra,runout,Abhi Vyas,NaN,NaN,NaN,2,5.0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
3,Nikhil Gupta Dabangg,bowled,NaN,Shankar VK,NaN,NaN,33,45.0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
4,Hari Chockalingam,not out,NaN,NaN,NaN,NaN,5,5.0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1


In [343]:
df_player_bowling_scores.head()

,Bowler,Overs,Maiden,No_Balls,Wide,Runs,Wicket,match_id,league_id,season_id,match_date,opposition,ground
0,Sankar Gudapati,2.0,0.0,NaN,NaN,11,0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
1,Harsha Gorla,2.0,0.0,NaN,NaN,14,1,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
2,Shankar VK,3.0,0.0,NaN,NaN,9,1,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
3,Srinivas Potluri,4.0,0.0,NaN,NaN,19,0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
4,Sudheer Palempati,4.0,0.0,NaN,NaN,18,0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1


In [344]:
final_df_batting = df_team_players_batting[['Player','Player_Id']].merge(df_player_batting_scores, left_on = 'Player', right_on = 'Batsman', how = 'inner')
final_df_batting[['Sixs','Fours']] = final_df_batting[['Sixs','Fours']].fillna(0)
final_df_batting.head()

,Player,Player_Id,Batsman,How_out,Fielder,Bowler,Sixs,Fours,Runs,Balls,match_id,league_id,season_id,match_date,opposition,ground
0,Sampath Bodduluri,62464,Sampath Bodduluri,bowled,NaN,Nikhil Gupta Dabangg,0.0,0.0,8,16.0,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
1,Sampath Bodduluri,62464,Sampath Bodduluri,caught,Raghav BATMEN,Neerav Basant,0.0,0.0,8,11.0,19410,4,56,2022-05-01,BATMEN,North SeaTac Park
2,Sampath Bodduluri,62464,Sampath Bodduluri,caught,RadhaKrishna Karri,Praneeth K,2.0,0.0,39,46.0,19470,4,56,2022-05-15,Team-MIB,North SeaTac Park
3,Sampath Bodduluri,62464,Sampath Bodduluri,caught,Kiran Modak,Harsha Reddy,0.0,0.0,0,1.0,19789,4,56,2022-05-21,Galacticos,Eastlake High School Stadium
4,Sampath Bodduluri,62464,Sampath Bodduluri,bowled,NaN,Siva Chaganti,0.0,0.0,2,3.0,19566,4,56,2022-06-18,DeadPool,GrassLawn Park-Softball Field 1


In [345]:
final_df_bowling = df_team_players_bowling[['Player','Player_Id']].merge(df_player_bowling_scores, left_on = 'Player', right_on = 'Bowler', how = 'inner')
# final_df[['Sixs','Fours']] = final_df[['Sixs','Fours']].fillna(0)
final_df_bowling.head()

,Player,Player_Id,Bowler,Overs,Maiden,No_Balls,Wide,Runs,Wicket,match_id,league_id,season_id,match_date,opposition,ground
0,Harsha Gorla,62634,Harsha Gorla,2.0,0.0,NaN,NaN,14,1,19237,4,56,2022-03-26,Dabangg Regiment,GrassLawn Park-Softball Field 1
1,Harsha Gorla,62634,Harsha Gorla,4.0,0.0,NaN,3.0,12,3,19410,4,56,2022-05-01,BATMEN,North SeaTac Park
2,Harsha Gorla,62634,Harsha Gorla,3.0,2.0,NaN,NaN,1,2,19470,4,56,2022-05-15,Team-MIB,North SeaTac Park
3,Harsha Gorla,62634,Harsha Gorla,3.0,0.0,NaN,4.0,22,3,19789,4,56,2022-05-21,Galacticos,Eastlake High School Stadium
4,Harsha Gorla,62634,Harsha Gorla,3.0,0.0,NaN,NaN,10,0,19566,4,56,2022-06-18,DeadPool,GrassLawn Park-Softball Field 1


In [346]:
# import seaborn as sns
# sns.histplot(data=final_df, x="Player", y ="Runs", hue="Player", multiple="dodge", shrink=.8)
# sns.histplot(data=final_df, x="Player", y ="Runs", alpha = .7)

# final_df.plot(x = "match_id", y=["Runs", "Balls"], kind="bar", figsize=(20, 8) )

In [347]:
pivot_ui(final_df_batting, rows=['match_date'], cols=['Player'], vals=['Runs'], aggregatorName='Sum', rendererName='Bar Chart', menuLimit=50, showUI = False, outfile_path="batting_runs_scored.html")

In [348]:
pivot_ui(final_df_batting, rows=['Player'], cols=['How_out'], vals=['Runs'], aggregatorName='Count', rendererName='Row Heatmap', menuLimit=50, showUI = True, outfile_path="batting_how_out.html")

In [327]:
pivot_ui(final_df_bowling, rows=['match_date'], cols=['Player'], vals=['Runs'], aggregatorName='Sum', rendererName='Bar Chart', menuLimit=50, showUI = False, outfile_path="bowling_runs_given.html")

In [328]:
pivot_ui(final_df_bowling, rows=['match_date'], cols=['Player'], vals=['Wicket'], aggregatorName='Sum', rendererName='Bar Chart', menuLimit=50, showUI = False, outfile_path="bowling_wickets_taken.html")

In [329]:
# %%javascript
# var e = document.createElement('script'); e.setAttribute('src', 'https://nytimes.github.io/svg-crowbar/svg-crowbar.js'); e.setAttribute('class', 'svg-crowbar'); document.body.appendChild(e);